In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import wbgapi as wb

pd.options.display.float_format = '{:,.2f}'.format

## plotly express with pandas

In [13]:
import plotly.express as px

indicator_code = "SL.UEM.TOTL.NE.ZS"
country_list = ["AUS", "ARB", "SAS"]

def get_data(country_list, indicator_code):
    """fetches data from worldbank by indicator code and country list from 1990 onwards"""
    df = wb.data.DataFrame(indicator_code, country_list, mrv=33)
    df.columns = [col.replace("YR", "") for col in df.columns]
    df.reset_index(inplace=True)
    df = df.melt(id_vars="economy", var_name="Year", value_name="y").sort_values(
        "economy"
    )
    return df


# rename region code to propper region name
country_codes = pd.read_csv("data/country_codes.csv")
country_code_dict = pd.Series(
    country_codes.value.values, index=country_codes.id
).to_dict()


def resolve_country_code(code):
    region = country_code_dict[code]
    return region


def plot(country_list, indicator_code):
    """plots indicator given by indicator coe and country list"""
    df = get_data(country_list, indicator_code)
    indicator_name = wb.series.get(indicator_code)["value"]

    # Create the line plot using Plotly Express
    fig = px.line(
        df.sort_values("Year"),
        x="Year",
        y=df.y,
        color="economy",
        title=indicator_name,
    )

    # Customize the layout
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="",
        legend_title_text="",
        hovermode="closest",  # Change how the hover information is displayed
        showlegend=True,  # Show legend
        template="plotly_white",  # Set plot template to white background
        title=dict(
            font=dict(size=18),
            x=0.5,
            y=0.92,
            xanchor="center",
            yanchor="top",
        ),
        legend=dict(
            orientation="h",
            xanchor="left",
            yanchor="bottom",
            x=0,
            y=1.02,
            groupclick="toggleitem",
            font=dict(family="Courier", size=12, color="black"),
        ),
        xaxis=dict(
            tickmode="linear",
            dtick=5,  # Show every fifth year
        ),
        modebar_orientation="v",
        modebar_remove=["zoom", "pan", "displaylogo"],
    )

    # Replace region codes with region names in legend labels
    for trace in fig.data:
        trace_name = resolve_country_code(trace.name)
        trace.name = trace_name

    return fig


plot(country_list, indicator_code)